In [3]:
import pandas as pd
import unicodedata
from nltk.corpus import stopwords
import nltk
import re
import math
import spotlight
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer 
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tag import StanfordNERTagger
from nltk.tokenize import word_tokenize
from io import StringIO
import os
import spacy
from spacy import displacy
from collections import Counter
from pprint import pprint
import en_core_web_sm

java_path = "C:\\Program Files\\Java\\jdk1.8.0_231\\bin\\java.exe"
os.environ['JAVAHOME'] = java_path

nlp = en_core_web_sm.load()
nltk.download('wordnet')
nltk.download("stopwords")
lemmatizer = WordNetLemmatizer()
pd.set_option('display.max_colwidth', -1)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\kadss\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kadss\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
# file load, cleaning and unicode conversion

file_name='Boston2C.csv'

for_pd = StringIO()
with open('Data/2CVTweets/'+file_name,encoding="utf8") as f:
    for line in f:
        new_line = re.sub(r'NO;', 'NO',line)  ##cleaning 'NO'
        print (new_line, file=for_pd)

for_pd.seek(0)

df_city = pd.read_csv(for_pd, sep=';', header=None)
df_city = df_city.drop(df_city.columns[0], axis=1) ## index column
df_city.columns=['tweet','Y']


In [8]:
df_city.head(20)

,tweet,Y
0,"RT @CommuterBoston: ACCIDENT (Wellesley, MA): RT-9 East near Cedar St - involving a motorcycle and a car - possible lane restrictions",YES
1,RT @CommuterBoston: ACCIDENT: I-290 East before I-495 (Exit #26) - 2-car accident with possible lane blockages,YES
2,Opened report via iPhone at 19 Standard St http://t.co/8vc761aCHT. Road kill on sidewalk.,NO
3,RT @CommuterBoston: ACCIDENT REPORTED: I-93 North before Granite Av (Exit #11) - police investigating,YES
4,Duduk lebih dari 7 jam sehari berisiko depresi 47 persen lebih tinggi dibandingkan yang hanya duduk 4 jam sehari [HuffingtonPost],NO
5,"RT @CommuterBoston: ACCIDENT (Providence, RI): I-195 West near Gano St - right shoulder blocked",YES
6,Dedham Police Looking for Suspects After Car Crashes into Verizon Store http://t.co/u3zrVjliqB,YES
7,@NotifyBoston We need a don''t block the box ordinance... And traffic needs to be fixed on Atlantic Ave btwn SStation and the offramps,NO
8,# T-MOBILE NOWE HORYZONTY: Punk''s not dead - Wielkie oglądanie filmów we Wrocławiu trwa w najleps... http://t.co/UhtgxQES8x,NO
9,"ACCIDENT REPORTED (Danvers, MA): RT-1 North off-ramp to I-95 North - use caution",YES


In [ ]:
# slang conversion

In [15]:
slang_covert_url='http://www.netlingo.com/acronyms.php'
slangdict= {}


from bs4 import BeautifulSoup
import requests, json
resp = requests.get('http://www.netlingo.com/acronyms.php')
soup = BeautifulSoup(resp.text, "html.parser")

key=""
value=""
for div in soup.findAll('div', attrs={'class':'list_box3'}):
    
    for li in div.findAll('li'):
        for a in li.findAll('a'):
            key =a.text
            value = li.text.split(key)[1]
            slangdict[key.lower()]=value
            
def slang_to_formal(input):
    i=1
    formal=[]
    for slang in input.split():
        i=i+1
        if slang.lower() in slangdict:
            formal.append(slangdict[slang])
        elif slang.isdigit():
            
            formal.append('D')  #not working
        else:
            formal.append(slang)
       
        
    return  " ".join(str(x) for x in formal)
            
        

In [17]:
df_city['tweet'] = df_city.apply(lambda row: slang_to_formal(row['tweet']), axis=1)

KeyError: ('RT', 'occurred at index 0')

In [18]:
slangdict

{'!': 'I have a comment',
 '#ff': 'Follow Friday',
 '(u)': 'it means arms around you, hug for you',
 '*$': 'Starbucks',
 '**//': 'it means wink wink, nudge nudge',
 ',!!!!': 'Talk to the hand',
 '/r/': 'Requesting',
 '02': 'Your (or my) two cents worth, also seen as m.',
 '10q': 'Thank you',
 '1174': 'Nude club',
 '121': 'One to one',
 '123': 'it means I agree',
 '1337': 'Elite -or- leet -or- L337',
 '14': 'it refers to the fourteen words',
 '142n8ly': 'Unfortunately',
 '143': 'I love you',
 '1432': 'I Love You Too',
 '14aa41': 'One for All and All for One',
 '182': 'I hate you',
 '187': 'it means murder/ homicide',
 '19': '0 hand',
 '1daful': 'it means wonderful',
 '1v4': 'One Vs. Four',
 '2': 'it means to, too, two',
 '20': 'Location',
 '24/7': 'Twenty Four Seven, as in all the time',
 '2b': 'To be',
 '2b or not 2b': 'To Be Or Not To Be',
 '2b@': 'To Be At',
 '2bz4uqt': 'Too Busy For You Cutey',
 '2b~not2b': 'To be or not to be',
 '2d4': 'To die for',
 '2day': 'Today',
 '2dloo': 'Too